In [11]:
#%load_ext tensorboard
#%tensorboard --logdir 'MNIST/runs' --port 6006

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms.v2 as tfs
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torchvision.datasets import MNIST
from torch.utils.data import random_split

from torch.utils.tensorboard import SummaryWriter

import os
import json
from PIL import Image


In [12]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

os.system("rm -rf runs")

writer = SummaryWriter("/Users/macbook/Documents/Документы — gwyndolin/GitHub/dl/MNIST/runs")

class DigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128, bias=False)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10)
        self.bn1 = nn.BatchNorm1d(128)
        #self.bn2 = nn.BatchNorm1d(64)
        
        
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x)) 
        #x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x        

mps


In [13]:
train_transform = tfs.Compose([
    tfs.ToImage(),
    tfs.Pad(4),
    tfs.RandomCrop(28),
    #tfs.RandomAffine(degrees=(-45, 45), translate=(0.1, 0.1), scale=(0.9, 1.1)),
    #tfs.RandomPerspective(distortion_scale=0.2, p=0.5),
    tfs.RandomAffine(degrees=(-10,10), translate=(0.05,0.05), scale=(0.95,1.05)),
    tfs.RandomPerspective(distortion_scale=0.1, p=0.3),
    tfs.ToDtype(torch.float32, scale=True),
    tfs.Normalize((0.5,), (0.5,))
])

val_transform = tfs.Compose([
    tfs.ToImage(),
    tfs.ToDtype(torch.float32, scale=True),
    tfs.Normalize((0.5,), (0.5,))
])

train_dataset_full = MNIST(
    root="./data",
    train=True,
    transform=train_transform,
    download=True
)

validate_dataset_full = MNIST(
    root="./data",
    train=True,
    transform=val_transform,
    download=True
)

train_len = int(0.7 * len(train_dataset_full))
val_len = len(train_dataset_full) - train_len

torch.manual_seed(42)
train_dataset, _ = random_split(train_dataset_full, [train_len, val_len])

torch.manual_seed(42)
_, validate_dataset = random_split(validate_dataset_full, [train_len, val_len])

train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
validate_loader = data.DataLoader(validate_dataset, batch_size=128, shuffle=False)

model = DigitModel().to(device)
model.train()

lr = 0.001
epochs = 80
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

In [ ]:
os.system("rm -rf MNIST/checkpoints")
os.makedirs("MNIST/checkpoints", exist_ok=True)

for epoch in range(epochs):
    # ---------- TRAIN ----------
    model.train()
    train_tqdm = tqdm(train_loader, leave=True)
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for i, (x, y) in enumerate(train_tqdm):
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_func(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # статистика
        train_loss += loss.item()
        preds = y_pred.argmax(dim=1)
        train_correct += (preds == y).sum().item()
        train_total += len(y)

        avg_train_loss = train_loss / (i + 1)
        train_acc = train_correct / train_total

        train_tqdm.set_description(f"Epoch {epoch+1}/{epochs} | Loss: {avg_train_loss:.4f} | Acc: {train_acc:.4f}")

    # ---------- VALIDATION ----------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        validate_tqdm = tqdm(validate_loader, leave=True)
        for i, (x, y) in enumerate(validate_tqdm):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_func(y_pred, y)

            val_loss += loss.item()
            preds = y_pred.argmax(dim=1)
            val_correct += (preds == y).sum().item()
            val_total += len(y)

            avg_val_loss = val_loss / (i + 1)
            batch_acc = val_correct / val_total
            validate_tqdm.set_description(f"Val | Loss: {avg_val_loss:.4f} | Acc: {batch_acc:.4f}")

    # ---------- Metrics ----------
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(validate_loader)
    train_acc = train_correct / train_total
    val_acc = val_correct / val_total

    # ---------- TensorBoard ----------
    writer.add_scalar("Loss/train", avg_train_loss, epoch)
    writer.add_scalar("Loss/val", avg_val_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Accuracy/val", val_acc, epoch)
    writer.flush()

    # ---------- Checkpoints ----------
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f"MNIST/checkpoints/mnist_epoch_{epoch}.pt")

    #print(f"\n✅ Epoch {epoch+1}/{epochs} | "
    #      f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
    #      f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}\n")

Epoch 1/80 | Loss: 1.1551 | Acc: 0.6143: 100%|██████████| 657/657 [00:14<00:00, 45.37it/s]
Val | Loss: 0.5719 | Acc: 0.8216: 100%|██████████| 141/141 [00:02<00:00, 68.84it/s]
Epoch 2/80 | Loss: 0.6755 | Acc: 0.7814: 100%|██████████| 657/657 [00:11<00:00, 56.63it/s]
Val | Loss: 0.3970 | Acc: 0.8798: 100%|██████████| 141/141 [00:02<00:00, 58.71it/s]
Epoch 3/80 | Loss: 0.5421 | Acc: 0.8251: 100%|██████████| 657/657 [00:15<00:00, 41.32it/s]
Val | Loss: 0.3443 | Acc: 0.8902: 100%|██████████| 141/141 [00:02<00:00, 63.99it/s]
Epoch 4/80 | Loss: 0.4822 | Acc: 0.8459: 100%|██████████| 657/657 [00:14<00:00, 44.89it/s]
Val | Loss: 0.3018 | Acc: 0.9043: 100%|██████████| 141/141 [00:03<00:00, 45.31it/s]
Epoch 5/80 | Loss: 0.4360 | Acc: 0.8618: 100%|██████████| 657/657 [00:16<00:00, 38.70it/s]
Val | Loss: 0.2661 | Acc: 0.9153: 100%|██████████| 141/141 [00:03<00:00, 46.19it/s]
Epoch 6/80 | Loss: 0.4071 | Acc: 0.8707: 100%|██████████| 657/657 [00:15<00:00, 41.87it/s]
Val | Loss: 0.2620 | Acc: 0.9168: 

In [1]:
writer.close()

NameError: name 'writer' is not defined

In [ ]:
state_dict = model.state_dict()
torch.save(state_dict, "MNIST/finish/mnist.tar")
writer.close()

In [16]:
# --- Трансформации для теста ---
test_transform = tfs.Compose([
    tfs.ToImage(),
    tfs.ToDtype(torch.float32, scale=True),
    tfs.Normalize((0.5,), (0.5,))
])

# --- MNIST test (автоматическая загрузка) ---
test_d = MNIST(
    root="./data",
    train=False,
    transform=test_transform,
    download=True
)

# --- DataLoader ---
test_data = data.DataLoader(test_d, batch_size=500, shuffle=False)

# --- Оценка точности ---
model.eval()
Q = 0

with torch.no_grad():
    for x, y in test_data:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        y_pred = torch.argmax(y_pred, dim=1)
        Q += (y_pred == y).sum().item()

Q /= len(test_d)
print(f"Accuracy: {Q:.4f}")

Accuracy: 0.9714
